In [1]:
import torch

In [2]:
import numpy as np

In [3]:
from torch import nn
import torch.optim as optim

In [4]:
import os

In [5]:
            
class Model(torch.nn.Module):
    def __init__(self, gen_emb, domain_emb, num_classes=3, dropout=0.5, crf=False):
        super(Model, self).__init__()
        self.gen_embedding = torch.nn.Embedding(gen_emb.shape[0], gen_emb.shape[1])
        self.gen_embedding.weight=torch.nn.Parameter(torch.from_numpy(gen_emb), requires_grad=False)
        self.domain_embedding = torch.nn.Embedding(domain_emb.shape[0], domain_emb.shape[1])
        self.domain_embedding.weight=torch.nn.Parameter(torch.from_numpy(domain_emb), requires_grad=False)
    
        self.conv1=torch.nn.Conv1d(gen_emb.shape[1]+domain_emb.shape[1], 128, 5, padding=2 )
        self.conv2=torch.nn.Conv1d(gen_emb.shape[1]+domain_emb.shape[1], 128, 3, padding=1 )
        self.dropout=torch.nn.Dropout(dropout)

        self.conv3=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv4=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv5=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.linear_ae=torch.nn.Linear(256, num_classes)
        self.crf_flag=crf
        if self.crf_flag:
            from allennlp.modules import ConditionalRandomField
            self.crf=ConditionalRandomField(num_classes)            
          
    def forward(self, x, x_len, x_mask, x_tag=None, testing=False):
        x_emb=torch.cat((self.gen_embedding(x), self.domain_embedding(x) ), dim=2)
        x_emb=self.dropout(x_emb).transpose(1, 2)
        x_conv=torch.nn.functional.relu(torch.cat((self.conv1(x_emb), self.conv2(x_emb)), dim=1) )
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv3(x_conv) )
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv4(x_conv) )
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv5(x_conv) )
        x_conv=x_conv.transpose(1, 2)
        x_logit=self.linear_ae(x_conv)
        if testing:
            if self.crf_flag:
                score=self.crf.viterbi_tags(x_logit, x_mask)
            else:
                x_logit=x_logit.transpose(2, 0)
                score=torch.nn.functional.log_softmax(x_logit).transpose(2, 0)
        else:
            if self.crf_flag:
                score=-self.crf(x_logit, x_tag, x_mask)
            else:
                x_logit=torch.nn.utils.rnn.pack_padded_sequence(x_logit, x_len, batch_first=True)
                score=torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(x_logit.data), x_tag.data)
        return score

In [ ]:
cd global_laptop/

In [1]:
#cd ../

In [ ]:
cd 95%/

In [133]:
model = torch.load('laptop0_global.pt') # laptop1

In [134]:
# Full precision model's parameters after pruning
params = 0
p = []
for i in model.state_dict().keys():
  imd = (model.state_dict()[i] !=0.).sum().item()
  print(i,imd)
  p.append(imd)
  params += imd

print("\nTotal parameters : ",params)

gen_embedding.weight 154479
domain_embedding.weight 37586
conv1.bias 0
conv1.weight 0
conv2.bias 0
conv2.weight 0
conv3.bias 0
conv3.weight 0
conv4.bias 0
conv4.weight 0
conv5.bias 0
conv5.weight 0
linear_ae.bias 0
linear_ae.weight 0

Total parameters :  192065


In [135]:
(4505835 - params)/4505835 # Laptop

0.9573741603942444

In [ ]:
(4612835 - params)/4612835 # Restaurant

In [136]:
#p1 = [2449800, 768600, 128, 256000, 128, 153600, 256,327680, 256, 327680, 256, 327680, 3, 768] # Restaurant
p1 = [2449800, 661600, 128, 256000, 128, 153600, 256,327680, 256, 327680, 256, 327680, 3, 768]  # Laptop

params = 0
p2 = []
for i,j,k in zip(model.state_dict().keys(), p, p1):
    if 'bias' in i:
        print(i," : ", "   Pruned :" ,j, "FP : ", k, " >>> ",(k-j)/k)
    else:
        print(i," : ", " Pruned :" ,j, "FP : ", k, " >>> ",(k-j)/k)
    imd = (model.state_dict()[i] !=0.).sum().item()
    p2.append(imd)
    params += imd
print()
print("Total parameters of FP: " , sum(p1), " Pruned :", params, " >>> ",(sum(p1)-params)/sum(p1))

gen_embedding.weight  :   Pruned : 154479 FP :  2449800  >>>  0.9369421993632133
domain_embedding.weight  :   Pruned : 37586 FP :  661600  >>>  0.943189238210399
conv1.bias  :     Pruned : 0 FP :  128  >>>  1.0
conv1.weight  :   Pruned : 0 FP :  256000  >>>  1.0
conv2.bias  :     Pruned : 0 FP :  128  >>>  1.0
conv2.weight  :   Pruned : 0 FP :  153600  >>>  1.0
conv3.bias  :     Pruned : 0 FP :  256  >>>  1.0
conv3.weight  :   Pruned : 0 FP :  327680  >>>  1.0
conv4.bias  :     Pruned : 0 FP :  256  >>>  1.0
conv4.weight  :   Pruned : 0 FP :  327680  >>>  1.0
conv5.bias  :     Pruned : 0 FP :  256  >>>  1.0
conv5.weight  :   Pruned : 0 FP :  327680  >>>  1.0
linear_ae.bias  :     Pruned : 0 FP :  3  >>>  1.0
linear_ae.weight  :   Pruned : 0 FP :  768  >>>  1.0

Total parameters of FP:  4505835  Pruned : 192065  >>>  0.9573741603942444
